In [1]:
import pandas as ps
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score , precision_score , classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler , OrdinalEncoder , LabelEncoder
from sklearn.impute import SimpleImputer

In [2]:
df = sns.load_dataset('titanic')
df

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [3]:
df.shape

(891, 15)

In [4]:
df.columns

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone'],
      dtype='object')

In [5]:
df.drop(['alive' , 'embark_town' , 'class' , 'deck' , 'adult_male' , 'alone'] , axis=1 , inplace=True)

In [6]:
df

,survived,pclass,sex,age,sibsp,parch,fare,embarked,who
0,0,3,male,22.0,1,0,7.2500,S,man
1,1,1,female,38.0,1,0,71.2833,C,woman
2,1,3,female,26.0,0,0,7.9250,S,woman
3,1,1,female,35.0,1,0,53.1000,S,woman
4,0,3,male,35.0,0,0,8.0500,S,man
...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,man
887,1,1,female,19.0,0,0,30.0000,S,woman
888,0,3,female,NaN,1,2,23.4500,S,woman
889,1,1,male,26.0,0,0,30.0000,C,man


In [7]:
df.isna().sum()

survived      0
pclass        0
sex           0
age         177
sibsp         0
parch         0
fare          0
embarked      2
who           0
dtype: int64

In [8]:
df.duplicated().sum()

111

In [9]:
df.drop_duplicates(inplace=True)

In [10]:
df.duplicated().sum()

0

In [11]:
df.dtypes

survived      int64
pclass        int64
sex          object
age         float64
sibsp         int64
parch         int64
fare        float64
embarked     object
who          object
dtype: object

In [12]:
df[['survived' , 'pclass' , 'sibsp']] = df[['survived' , 'pclass' , 'sibsp']].astype('object')

In [13]:
df.dtypes

survived     object
pclass       object
sex          object
age         float64
sibsp        object
parch         int64
fare        float64
embarked     object
who          object
dtype: object

In [14]:
X = df.drop('survived' , axis=1)
y = df['survived']
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2,random_state=23)

In [15]:
cat_col = X_train.select_dtypes(include='object').columns
num_col = X_train.select_dtypes(exclude='object').columns

In [16]:
num_col

Index(['age', 'parch', 'fare'], dtype='object')

**Pipline**

In [17]:
cat_transformer = Pipeline(steps=[('Mode Imputation' , SimpleImputer(strategy='most_frequent')),
                                  ('Encoding' , OrdinalEncoder())])
cat_transformer

Pipeline(steps=[('Mode Imputation', SimpleImputer(strategy='most_frequent')),
                ('Encoding', OrdinalEncoder())])

In [18]:
num_transformer = Pipeline(steps=[('Mean Imputation' , SimpleImputer(strategy='mean')),
                                  ('Scaling ' , StandardScaler())])
num_transformer

Pipeline(steps=[('Mean Imputation', SimpleImputer()),
                ('Scaling ', StandardScaler())])

In [19]:
preprocessing = ColumnTransformer([('step1' , cat_transformer , cat_col),
                   ('step2' , num_transformer , num_col)] ,
                    remainder='passthrough')

preprocessing

ColumnTransformer(remainder='passthrough',
                  transformers=[('step1',
                                 Pipeline(steps=[('Mode Imputation',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('Encoding',
                                                  OrdinalEncoder())]),
                                 Index(['pclass', 'sex', 'sibsp', 'embarked', 'who'], dtype='object')),
                                ('step2',
                                 Pipeline(steps=[('Mean Imputation',
                                                  SimpleImputer()),
                                                 ('Scaling ',
                                                  StandardScaler())]),
                                 Index(['age', 'parch', 'fare'], dtype='object'))])

In [20]:
pipe = Pipeline([('Preprocessing' , preprocessing),
          ('Classification' , KNeighborsClassifier())])

pipe

Pipeline(steps=[('Preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('step1',
                                                  Pipeline(steps=[('Mode '
                                                                   'Imputation',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('Encoding',
                                                                   OrdinalEncoder())]),
                                                  Index(['pclass', 'sex', 'sibsp', 'embarked', 'who'], dtype='object')),
                                                 ('step2',
                                                  Pipeline(steps=[('Mean '
                                                                   'Imputation',
                                                                   SimpleImputer()),
                                                                  ('Scaling ',
                                                                   StandardScaler())]),
                                                  Index(['age', 'parch', 'fare'], dtype='object'))])),
                ('Classification', KNeighborsClassifier())])

In [21]:
pipe.fit(X_train , y_train)

ValueError: Unknown label type: unknown. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.

In [ ]:
y_test = y_test.astype('int')
y_train = y_train.astype('int')

In [ ]:
pipe.fit(X_train , y_train)

In [ ]:
y_pred = pipe.predict(X_test)
y_pred

In [ ]:
accuracy_score(y_test , y_pred)

In [ ]:
print(classification_report(y_test , y_pred))

**Combine pipline code**

In [ ]:
cat_transformer = Pipeline(steps=[('Mode Imputation' , SimpleImputer(strategy='most_frequent')),
                                  ('Encoding' , OrdinalEncoder())])

num_transformer = Pipeline(steps=[('Mean Imputation' , SimpleImputer(strategy='mean')),
                                  ('Scaling ' , StandardScaler())])

preprocessing = ColumnTransformer([('step1' , cat_transformer , cat_col),
                   ('step2' , num_transformer , num_col)] ,
                    remainder='passthrough')

pipe = Pipeline([('Preprocessing' , preprocessing),
          ('Classification' , KNeighborsClassifier())])

pipe